<a href="https://colab.research.google.com/github/ralsouza/apache_spark_real_time_analytics/blob/master/notebooks/03_pyspark_project_uber_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Setup

In [2]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [3]:
# Install the dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
# Environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
# Libraries and Context Setup
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
sqlContext = pyspark.SQLContext(sc)

In [6]:
# Check context
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [ ]:
# sc.stop()

# Mini-Projeto 1 - Analisando Dados do Uber com Spark 

Dataset: https://github.com/fivethirtyeight/uber-tlc-foil-response

Esse conjunto de dados contém dados de mais de 4,5 milhões de captações Uber na cidade de Nova York de abril a setembro de 2014 e 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015.

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passaram pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

In [7]:
from pandas import read_csv

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
# Criando um objeto Pandas
uberFile = read_csv('/content/drive/My Drive/Colab Notebooks/08-apache-spark/data/uber.csv')

In [10]:
type(uberFile)

pandas.core.frame.DataFrame

In [ ]:
# Visualizando as primeiras linhas
uberFile.head(10)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537
5,B02598,1/1/2015,870,6903
6,B02598,1/2/2015,785,4768
7,B02617,1/2/2015,1137,7065
8,B02512,1/2/2015,175,875
9,B02682,1/2/2015,890,5506


In [11]:
# Tranformando o dataframe (Pandas) em um Dataframe (Spark)
df_uber = spark.createDataFrame(uberFile)

In [ ]:
type(df_uber)

pyspark.sql.dataframe.DataFrame

In [ ]:
type(df_uber)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Total rows
df_uber.count()

354

In [ ]:
# First row
df_uber.printSchema()

root
 |-- dispatching_base_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- active_vehicles: long (nullable = true)
 |-- trips: long (nullable = true)



In [ ]:
# Sample data
df_uber.show()

+-----------------------+--------+---------------+-----+
|dispatching_base_number|    date|active_vehicles|trips|
+-----------------------+--------+---------------+-----+
|                 B02512|1/1/2015|            190| 1132|
|                 B02765|1/1/2015|            225| 1765|
|                 B02764|1/1/2015|           3427|29421|
|                 B02682|1/1/2015|            945| 7679|
|                 B02617|1/1/2015|           1228| 9537|
|                 B02598|1/1/2015|            870| 6903|
|                 B02598|1/2/2015|            785| 4768|
|                 B02617|1/2/2015|           1137| 7065|
|                 B02512|1/2/2015|            175|  875|
|                 B02682|1/2/2015|            890| 5506|
|                 B02765|1/2/2015|            196| 1001|
|                 B02764|1/2/2015|           3147|19974|
|                 B02765|1/3/2015|            201| 1526|
|                 B02617|1/3/2015|           1188|10664|
|                 B02598|1/3/20

In [ ]:
# Convert csv file to rdd
uber_test = sc.textFile('/content/drive/My Drive/Colab Notebooks/08-apache-spark/data/uber.csv')
type(uber_test)

pyspark.rdd.RDD

# 1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

In [ ]:
df_uber_count = df_uber.groupBy('dispatching_base_number').count()

In [ ]:
df_uber_count.show()

+-----------------------+-----+
|dispatching_base_number|count|
+-----------------------+-----+
|                 B02512|   59|
|                 B02598|   59|
|                 B02682|   59|
|                 B02765|   59|
|                 B02617|   59|
|                 B02764|   59|
+-----------------------+-----+



Temos 6 bases do uber.

# 2- Qual o total de veículos que passaram pela base B02617?

In [13]:
df_uber.groupBy('dispatching_base_number').sum('active_vehicles') \
  .where(df_uber.dispatching_base_number == 'B02617') \
  .show()

+-----------------------+--------------------+
|dispatching_base_number|sum(active_vehicles)|
+-----------------------+--------------------+
|                 B02617|               79758|
+-----------------------+--------------------+



# 3- Qual o total de corridas por base? Apresente de forma decrescente.

In [45]:
df_uber.groupBy('dispatching_base_number') \
  .sum('trips') \
  .orderBy('sum(trips)',ascending=False) \
  .show()

+-----------------------+----------+
|dispatching_base_number|sum(trips)|
+-----------------------+----------+
|                 B02764|   1914449|
|                 B02617|    725025|
|                 B02682|    662509|
|                 B02598|    540791|
|                 B02765|    193670|
|                 B02512|     93786|
+-----------------------+----------+

